## **Extraction & Preparation** 

This notebook demonstrates data extraction from an open data portal through an API, with a focus on implementing pagination for complete retrieval.   
    
Additionally, it covers data preparation steps, such as eliminating null values, formatting columns, and standardizing values. 

### Pagination Parameters

- `$where`: Defines the conditions for data retrieval.
- `$limit`: Specifies the number of records to retrieve per request.
- `$offset`: Represents the starting point for retrieving records.

In [28]:
# import libraries
from io import StringIO
from fuzzywuzzy import fuzz, process
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"
import random
import requests

In [29]:
# define api endpoint and set pagination variables
url = "https://data.iowa.gov/resource/m3tr-qhgy.csv"
limit = 50000  
offset = 0

all_data = []

# apply pagination using loop until all rows are loaded
while True:
    parameters = {
        "$where": "date >= '2023-06-01T00:00:00'",
        "$limit": limit,
        "$offset": offset
    }

    response = requests.get(url, params = parameters)
    response.raise_for_status()

    data_chunk = pd.read_csv(StringIO(response.text))
    
    if data_chunk.empty:
        break  
    
    # append data chunks to list and increment offset by limit
    all_data.append(data_chunk)
    offset += limit

# concatenate list of data chunks into dataframe
df = pd.concat(all_data, ignore_index = True)

### Dataset Overview

Below is a concise overview of the raw dataset, featuring 24 columns and around 1.3 million rows.     
      
It includes the first five and last five rows, along with the column names.

In [30]:
# dimensions of dataframe
df.shape

(1341858, 24)

In [31]:
# display column names 
df.columns

Index(['invoice_line_no', 'date', 'store', 'name', 'address', 'city',
       'zipcode', 'store_location', 'county_number', 'county', 'category',
       'category_name', 'vendor_no', 'vendor_name', 'itemno', 'im_desc',
       'pack', 'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
       'sale_bottles', 'sale_dollars', 'sale_liters', 'sale_gallons'],
      dtype='object')

In [32]:
# display initial rows 
df.head()

,invoice_line_no,date,store,name,address,city,zipcode,store_location,county_number,county,...,itemno,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
0,INV-58964400021,2023-06-01T00:00:00.000,2536,HY-VEE (1530) / PLEASANT HILL,4815 MAPLE DRIVE,PLEASANT HILL,50317.0,POINT (-93.519382986 41.59708798),NaN,POLK,...,36305,HAWKEYE VODKA PET,12,750,3.50,5.25,12,63.00,9.00,2.37
1,INV-58966700122,2023-06-01T00:00:00.000,2191,KEOKUK SPIRITS,1013 MAIN,KEOKUK,52632.0,POINT (-91.387520009 40.400196002),NaN,LEE,...,89206,1800 REPOSADO,12,750,17.50,26.25,2,52.50,1.50,0.39
2,INV-58946000001,2023-06-01T00:00:00.000,5972,CASEY'S GENERAL STORE #1623 / INDIANOLA,607 N JEFFERSON WAY,INDIANOLA,50125.0,POINT (-93.55888002299997 41.366162985000074),NaN,WARREN,...,10802,CROWN ROYAL PEACH,12,750,17.49,26.24,12,314.88,9.00,2.37
3,INV-58969400045,2023-06-01T00:00:00.000,3663,WAL-MART 1496 / WATERLOO,1334 FLAMMANG DR,WATERLOO,50702.0,POINT (-92.330571959 42.458433012),NaN,BLACK HAWK,...,58838,JOSE CUERVO AUTHENTIC LIME MARGARITA,6,1750,8.74,13.11,12,157.32,21.00,5.54
4,INV-58940000046,2023-06-01T00:00:00.000,5086,"CENTRAL MART I, LLC.",2501 FLOYD BLVD,SIOUX CITY,51106.0,POINT (-96.39353902999994 42.50354428800006),NaN,WOODBURY,...,84226,99 STRAWBERRIES MINI,1,50,51.60,77.40,1,77.40,0.05,0.01


In [33]:
# display final rows 
df.tail()

,invoice_line_no,date,store,name,address,city,zipcode,store_location,county_number,county,...,itemno,im_desc,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
1341853,INV-64789900009,2023-11-30T00:00:00.000,4668,CLEAR LAKE PAYLESS FOODS,20 SOUTH 4TH ST,CLEARLAKE,50428.0,POINT (-93.380110037 43.136112021),NaN,CERRO GORDO,...,38178,TITOS HANDMADE VODKA,6,1750,19.00,28.50,6,171.00,10.50,2.77
1341854,INV-64797500005,2023-11-30T00:00:00.000,4941,KUM & GO #170 / URBANDALE,6990 DOUGLAS AVE,URBANDALE,50322.0,POINT (-93.711711119 41.629104797),NaN,POLK,...,10805,CROWN ROYAL REGAL APPLE,24,375,8.49,12.74,6,76.44,2.25,0.59
1341855,INV-64801100004,2023-11-30T00:00:00.000,5484,CASEY'S GENERAL STORE #3610 / CEDAR FALLS,1225 FOUNTAINS WAY,CEDAR FALLS,50613.0,POINT (-92.460373022 42.498330997),NaN,BLACK HAWK,...,22215,WOODFORD RESERVE,12,750,23.50,35.25,12,423.00,9.00,2.37
1341856,INV-64786000015,2023-11-30T00:00:00.000,6129,EAST END LIQUOR / DES MOINES,3804 HUBBELL AVE,DES MOINES,50317.0,POINT (-93.541625959 41.631082004),NaN,POLK,...,27408,JIM BEAM HONEY MINI,12,50,7.00,10.50,1,10.50,0.05,0.01
1341857,INV-64793800091,2023-11-30T00:00:00.000,5651,SUPER QUICK MART / WINDSOR HEIGHTS,7690 HICKMAN RD,WINDSOR HEIGHTS,50324.0,POINT (-93.722437021 41.614007019),NaN,POLK,...,86887,SOUTHERN COMFORT,12,1000,12.97,19.46,2,38.92,2.00,0.52


## Data Processing
### Null Values

Null values are present in the six columns displayed below.           

Upon evaluating column importance, the store location column is unnecessary for the analysis and the county number column exclusively consists of null values, rendering it uninformative.             

Listwise deletion was applied to the remaining columns to drop 402 rows, which represents [less than 5% of the dataset](https://www.statisticssolutions.com/dissertation-resources/missing-values-in-data/#:~:text=Handling%20Missing%20Values&text=Suppose%20the%20number%20of%20cases,the%20researcher%20can%20drop%20them) (0.05%), ensuring minimal impact on future analysis.

In [34]:
# find columns with null values 
df.isnull().sum()[df.isnull().sum() > 0].reset_index()

,index,0
0,address,402
1,city,402
2,zipcode,402
3,store_location,1254
4,county_number,1341858
5,county,402


In [35]:
# drop rows with null values 
df = df[df['address'].notnull()]

In [36]:
# drop unecessary columns 
df = df.drop(columns = ['county_number','store_location'])

In [37]:
# validate dataframe dimensions
df.shape

(1341456, 22)

In [38]:
# validate removal of null values 
df.isnull().sum().reset_index()

,index,0
0,invoice_line_no,0
1,date,0
2,store,0
3,name,0
4,address,0
5,city,0
6,zipcode,0
7,county,0
8,category,0
9,category_name,0


### Column Formatting

There is a general lack of formatting within the dataset, so improvements were implemented to enhance cohesion.

This includes data type conversions, string formatting, and column name standardization for a more unified structure.

In [39]:
# format columns 

# string formatting
df.iloc[:,1:] = df.iloc[:,1:].applymap(lambda x: x.title() if isinstance(x, str) else x)

# data type conversion
df[['state_bottle_cost', 'state_bottle_retail', 'sale_dollars', 'sale_liters', 'sale_gallons']] = df[['state_bottle_cost', 'state_bottle_retail', 'sale_dollars', 'sale_liters', 'sale_gallons']].astype(float)
df[['store', 'zipcode', 'category', 'vendor_no', 'itemno', 'pack', 'bottle_volume_ml', 'sale_bottles']] = df[['store', 'zipcode', 'category', 'vendor_no', 'itemno', 'pack', 'bottle_volume_ml', 'sale_bottles']].astype(int)
df[['invoice_line_no', 'name', 'address', 'city', 'county', 'category_name',  'vendor_name', 'im_desc']] = df[['invoice_line_no', 'name', 'address', 'city', 'county', 'category_name',  'vendor_name', 'im_desc']].astype(str)
df['date'] = pd.to_datetime(df['date']).dt.date

# column name standardization
df.rename(columns = {'invoice_line_no': 'invoice_id', 'store':'store_id','name': 'store_name','zipcode':'zip_code','vendor_no':'vendor_id',
                     'itemno':'item_id','im_desc':'item_name','pack':'pack_size', 'category': 'category_id'}, inplace = True)

In [40]:
# validate column names 
df.columns

Index(['invoice_id', 'date', 'store_id', 'store_name', 'address', 'city',
       'zip_code', 'county', 'category_id', 'category_name', 'vendor_id',
       'vendor_name', 'item_id', 'item_name', 'pack_size', 'bottle_volume_ml',
       'state_bottle_cost', 'state_bottle_retail', 'sale_bottles',
       'sale_dollars', 'sale_liters', 'sale_gallons'],
      dtype='object')

In [41]:
# validate initial rows
df.head()

,invoice_id,date,store_id,store_name,address,city,zip_code,county,category_id,category_name,...,item_id,item_name,pack_size,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles,sale_dollars,sale_liters,sale_gallons
0,INV-58964400021,2023-06-01,2536,Hy-Vee (1530) / Pleasant Hill,4815 Maple Drive,Pleasant Hill,50317,Polk,1031100,American Vodkas,...,36305,Hawkeye Vodka Pet,12,750,3.50,5.25,12,63.00,9.00,2.37
1,INV-58966700122,2023-06-01,2191,Keokuk Spirits,1013 Main,Keokuk,52632,Lee,1022200,100% Agave Tequila,...,89206,1800 Reposado,12,750,17.50,26.25,2,52.50,1.50,0.39
2,INV-58946000001,2023-06-01,5972,Casey'S General Store #1623 / Indianola,607 N Jefferson Way,Indianola,50125,Warren,1012100,Canadian Whiskies,...,10802,Crown Royal Peach,12,750,17.49,26.24,12,314.88,9.00,2.37
3,INV-58969400045,2023-06-01,3663,Wal-Mart 1496 / Waterloo,1334 Flammang Dr,Waterloo,50702,Black Hawk,1071000,Cocktails/Rtd,...,58838,Jose Cuervo Authentic Lime Margarita,6,1750,8.74,13.11,12,157.32,21.00,5.54
4,INV-58940000046,2023-06-01,5086,"Central Mart I, Llc.",2501 Floyd Blvd,Sioux City,51106,Woodbury,1081400,American Schnapps,...,84226,99 Strawberries Mini,1,50,51.60,77.40,1,77.40,0.05,0.01


### Data Standardization

In the process of enhancing consistency, the following steps were implemented:

*   String cleaning and formatting were applied to the store name column.

*   Variants of the same city were identified and recoded to ensure uniformity - it was not necessary to identify similar counties as the dataset contains the 99 unique counties of Iowa.

*   Similarly, the category name and category identifier values have been recoded to consolidate product types, reducing overly detailed distinctions.  

*   The absence of a state column in the dataset led to the addition of a new column, assigning the two-letter state abbreviation of Iowa to each row.





#### Store Names

In [42]:
# display store names 
df['store_name'].value_counts().reset_index()

,store_name,count
0,Hy-Vee #3 / Bdi / Des Moines,10229
1,Central City 2,10209
2,Westside Spirits / Cedar Rapids,7738
3,Benz Distributing,7575
4,Hy-Vee Food Store / Cedar Falls,7455
...,...,...
2052,Templeton Distilling Llc (Et),3
2053,Tycoga Vineyard & Winery,3
2054,Wych Dr Distilling Co / Et,3
2055,Wych Dr Distilling Co / (Et),2


In [43]:
# clean and format store names 
df['store_name'] = df['store_name'].str.split("/").str[0]
df['store_name'] = df['store_name'].str.split(",").str[0]
df['store_name'] = df['store_name'].str.replace(r"  ", " ")
df['store_name'] = df['store_name'].str.replace(r"'S", "'s")
df['store_name'] = df['store_name'].str.replace(r"(Et)", "")

In [44]:
# validate clean store names 
df['store_name'].value_counts().reset_index() 

,store_name,count
0,Hy-Vee Food Store,45387
1,Hy-Vee Wine And Spirits,33794
2,Hy-Vee,25368
3,Hy-Vee Food Store #2,13943
4,Hy-Vee #3,13923
...,...,...
1931,173 Craft Distillery,4
1932,Mapleton Food Land,4
1933,Moes Mart #13,3
1934,Tycoga Vineyard & Winery,3


#### Cities

In [45]:
# identify unique pairs of similar city names 

# create list of unique cities
unique_cities = df['city'].unique()

# initialize set to store pairs
unique_similar_cities = set()

# iterate through unique cities and find similar ones
for city in unique_cities:
    matches = process.extract(city, unique_cities, limit = len(unique_cities), scorer = fuzz.ratio)
    similar_cities = [match[0] for match in matches if match[1] >= 85]
    if len(similar_cities) > 1:
        unique_similar_cities.add(tuple(sorted(similar_cities)))

# display unique similar city pairs
for similar_cities in unique_similar_cities:
    print(", ".join(similar_cities))

Pleasant Hill, Pleasantville
Mount Pleasant, Mt Pleasant
Saint Ansgar, St Ansgar
Aplington, Arlington
Pleasant Hill, Pleasant Valley, Pleasantville
Clear Lake, Clearlake
Le Claire, Leclaire
Monroe, Montrose
Creston, Preston
Pleasant Valley, Pleasantville


In [46]:
# select variants of multiple cities
selected_citites = ['Clear Lake', 'Clearlake', 'Le Claire', 'Leclaire', 'Saint Ansgar', 'St Ansgar', 'Mount Pleasant', 'Mt Pleasant']

# utilize mask to filter rows containing specified cities
mask = df['city'].isin(selected_citites)

# display value counts for specified cities
df[mask]['city'].value_counts().reset_index()

,city,count
0,Clearlake,4075
1,Clear Lake,4063
2,Mount Pleasant,3112
3,Le Claire,1840
4,Mt Pleasant,1375
5,St Ansgar,350
6,Saint Ansgar,130
7,Leclaire,27


In [47]:
# recode city column
cities = {
    'Clearlake': 'Clear Lake',
    'Leclaire': 'Le Claire',
    'St Ansgar': 'Saint Ansgar',
    'Mt Pleasant': 'Mount Pleasant'
}

# apply string replacement
for key, value in cities.items():
    df['city'] = df['city'].str.replace(key, value, case = False)

In [48]:
# validate consolidation of city values
df[mask]['city'].value_counts().reset_index()

,city,count
0,Clear Lake,8138
1,Mount Pleasant,4487
2,Le Claire,1867
3,Saint Ansgar,480


#### Categories

In [49]:
# display category names
df['category_name'].value_counts().reset_index()

,category_name,count
0,American Vodkas,212302
1,Canadian Whiskies,125481
2,Straight Bourbon Whiskies,100572
3,Whiskey Liqueur,84519
4,100% Agave Tequila,67573
5,American Flavored Vodka,65915
6,Spiced Rum,51006
7,Blended Whiskies,46370
8,American Schnapps,42310
9,Cocktails/Rtd,40897


In [50]:
# recode category id and category names columns

# define dictionary of strings and replacements 
categories = {
    'brandies':'Brandy',
    'gin':'Gin',
    'liqueur':'Liqueur',
    'rum':'Rum',
    'schnapps':'Schnapps',
    'tequila':'Tequila',
    'vodka':'Vodka',
    'whiskies|bourbon|scotch':'Whiskey'
}

# apply string replacement
conditions = [df['category_name'].str.contains(s, case = False) for s in categories.keys()]
choices = list(categories.values())
df['category_name'] = np.select(conditions, choices, default = 'Other')

# assign id values based on new categories 
category_ids = ['10' + str(random.randint(100, 999)).zfill(3) + '00' for _ in range(len(df['category_name'].unique()))]
category_mapping = dict(zip(df['category_name'].unique(), category_ids))
df['category_id'] = df['category_name'].map(category_mapping)


In [51]:
# validate new categories 
df['category_name'].value_counts().reset_index()

,category_name,count
0,Whiskey,371369
1,Vodka,324174
2,Liqueur,177278
3,Other,118513
4,Rum,111110
5,Tequila,97010
6,Schnapps,60115
7,Brandy,48834
8,Gin,33053


In [52]:
# validate new category identifiers
df['category_id'].value_counts().reset_index()

,category_id,count
0,1079900,371369
1,1052200,324174
2,1079400,177278
3,1020400,118513
4,1052800,111110
5,1095700,97010
6,1090000,60115
7,1010300,48834
8,1090200,33053


#### State

In [53]:
# create state column
df['state'] = 'IA'

## Data Exportation

The data is exported using the Parquet format, known for its columnar storage and compression advantages. 

This efficient storage facilitates seamless cross-notebook data transfer.

In [55]:
# export clean dataframe
df.to_parquet('../data/spirit_sales.parquet')